<a href="https://colab.research.google.com/github/Prasanthsai0987/Quantum-Machine-Learning-Tasks-/blob/main/QML_UseCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================================
# Quantum-Enhanced Machine Learning Model (QEML)
# Hybrid quantum-classical binary classifier using PennyLane
# ===========================================================

# --- INSTALL DEPENDENCIES ---
# (uncomment the following lines if running in Google Colab)
# !pip install pennylane scikit-learn numpy

# --- IMPORTS ---
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random

# --- FIX FOR RANDOMNESS / CLEAN START ---
random.seed(42)
np.random.seed(42)

# ===========================================================
# 1. DATA PREPARATION
# ===========================================================
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ===========================================================
# 2. QUANTUM CIRCUIT DESIGN
# ===========================================================
n_qubits = 2
n_layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

def angle_embedding(x):
    """Encodes classical features into qubit rotations."""
    for i in range(n_qubits):
        qml.RY(x[i % len(x)], wires=i)

def variational_layer(params):
    """Variational layer: RX-RY-RZ + entangling CNOTs."""
    for i in range(n_qubits):
        qml.RX(params[i, 0], wires=i)
        qml.RY(params[i, 1], wires=i)
        qml.RZ(params[i, 2], wires=i)
    # Simple entangling pattern
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

@qml.qnode(dev, interface="autograd")
def quantum_circuit(x, flat_params):
    """Full hybrid circuit returning expectation ⟨Z⟩ on qubit 0."""
    params = flat_params.reshape((n_layers, n_qubits, 3))
    angle_embedding(x)
    for l in range(n_layers):
        variational_layer(params[l])
    return qml.expval(qml.PauliZ(0))

# ===========================================================
# 3. HYBRID MODEL DEFINITIONS
# ===========================================================
def predict_prob(params, x):
    """Map expectation value → sigmoid probability."""
    expval = quantum_circuit(x, params)
    return 1 / (1 + np.exp(-expval))  # sigmoid

def loss_and_grad(params, X_batch, y_batch):
    """Compute batch loss + gradients."""
    def batch_loss(p):
        preds = np.array([predict_prob(p, x) for x in X_batch])
        eps = 1e-8
        preds = np.clip(preds, eps, 1 - eps)
        loss = -np.mean(y_batch * np.log(preds) + (1 - y_batch) * np.log(1 - preds))
        return loss
    loss_val = batch_loss(params)
    grad_val = qml.grad(batch_loss)(params)
    return loss_val, grad_val

# ===========================================================
# 4. TRAINING LOOP
# ===========================================================
num_params = n_layers * n_qubits * 3
params = 0.01 * np.random.randn(num_params, requires_grad=True)
epochs = 40
batch_size = 16
learning_rate = 0.1

for epoch in range(epochs):
    indices = np.arange(len(X_train))
    np.random.shuffle(indices)
    total_loss = 0.0

    for start in range(0, len(indices), batch_size):
        batch_idx = indices[start:start + batch_size]
        X_batch = X_train[batch_idx]
        y_batch = y_train[batch_idx]
        loss_val, grad_val = loss_and_grad(params, X_batch, y_batch)
        params = params - learning_rate * grad_val
        total_loss += loss_val * len(X_batch)

    avg_loss = total_loss / len(X_train)
    preds_train = [1 if predict_prob(params, x) >= 0.5 else 0 for x in X_train]
    train_acc = accuracy_score(y_train, preds_train)
    print(f"Epoch {epoch+1:02d}: Loss = {avg_loss:.4f}, Train Acc = {train_acc:.3f}")

# ===========================================================
# 5. EVALUATION
# ===========================================================
preds_test = [1 if predict_prob(params, x) >= 0.5 else 0 for x in X_test]
test_acc = accuracy_score(y_test, preds_test)
print(f"\nQuantum Model Test Accuracy: {test_acc:.3f}")

# ===========================================================
# 6. CLASSICAL BASELINE (LOGISTIC REGRESSION)
# ===========================================================
clf = LogisticRegression().fit(X_train, y_train)
base_acc = accuracy_score(y_test, clf.predict(X_test))
print(f"Classical Logistic Regression Test Accuracy: {base_acc:.3f}")


AttributeError: partially initialized module 'pennylane' has no attribute 'measurements' (most likely due to a circular import)

In [ ]:
pip install pennylane scikit-learn numpy


In [ ]:
pip install -U pennylane


In [ ]:
import pennylane as qml
print("✅ PennyLane imported successfully!")
print("Version:", qml.__version__)


AttributeError: partially initialized module 'pennylane' has no attribute 'measurements' (most likely due to a circular import)